In [1]:
# Get the risk_premium_market and residual_variance_market
import yfinance as yf
import numpy as np
import pandas as pd
import datetime

# Define start and end dates
end_date = datetime.datetime.now()
start_date = end_date - datetime.timedelta(days=5*365)

# Get the historical data for the S&P 500
sp500_data = yf.download('^GSPC', start=start_date, end=end_date)['Adj Close']

# Get the risk-free rate of return
risk_free_rate = 0.02  # Example risk-free rate of return (2%)

# Calculate the historical returns for the S&P 500
sp500_returns = sp500_data.pct_change().dropna()

# Calculate the average annual return of the S&P 500
average_return = sp500_returns.mean()

# Calculate the risk premium
risk_premium = average_return - risk_free_rate

# Calculate daily returns
sp500_data['Return'] = sp500_data.pct_change()

# Calculate standard deviation
std_dev = np.std(sp500_data['Return'])

residual_variance_market = (std_dev * np.sqrt(252))*(std_dev * np.sqrt(252))

# Calculate daily returns
risk_free_data = yf.download('^TNX', start=start_date, end=end_date)['Adj Close']
risk_free_returns = risk_free_data.pct_change().dropna()

# Calculate expected return and market risk premium
expected_return = sp500_returns.mean()
risk_premium_market = expected_return - risk_free_returns.mean()

print("Residual Variance Market: ", residual_variance_market)
print("Risk Premium Market: ", risk_premium_market)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Residual Variance Market:  0.04726906180402547
Risk Premium Market:  -0.0005215726435019651


C:\Users\mattg\AppData\Local\Temp\ipykernel_315056\149675599.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sp500_data['Return'] = sp500_data.pct_change()


In [2]:
# Random Forest
from pypfopt import plotting
import numpy as np
import pandas as pd
from sklearn.svm import SVR
from pypfopt import expected_returns, risk_models, EfficientFrontier
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestRegressor
import datetime
import yfinance as yf

# Define the ticker symbols for the market index and individual assets
market_index_ticker = "^GSPC"  # S&P 500 index
tickers = ['AAPL', 'AMD', 'AMZN', 'CCJ', 'COST', 'GOOG', 'GS', 'JPM', 'LLY', 'META', 'MSFT', 'NEE', 'PFE', 'SAP', 'WMT']

# Define the start and end dates for the historical data
end_date = datetime.datetime.now()
start_date = end_date - datetime.timedelta(days=5*365)

# Fetch the historical data using yfinance
data = yf.download([market_index_ticker] + tickers, start=start_date, end=end_date)

# Returns of stocks and market
market_index_returns = ((data['Close'][market_index_ticker] / data['Close'][market_index_ticker].shift(1))-1).dropna().tolist()
stock_returns = {}
for ticker in tickers:
    stock_returns[ticker] = ((data['Close'][ticker] / data['Close'][ticker].shift(1))-1).dropna().tolist()

# Future market returns prediction using logistic regression
market_x = np.array(market_index_returns[:-10]).reshape(-1, 1)
market_y = np.array(market_index_returns[10:])

reg = RandomForestRegressor()
reg.fit(market_x, market_y)
market_future = reg.predict(np.array(market_index_returns[-10:]).reshape(-1, 1))

# Fitting single index model regression on market return and stock return
single_index_regs = {}
betas = {}
alphas = {}
residual_variances = {}

for ticker in tickers:
    single_index_reg = LinearRegression()
    single_index_reg.fit(np.array(market_index_returns[:-10]).reshape(-1, 1), np.array(stock_returns[ticker][10:]).reshape(-1, 1))
    stock_future = single_index_reg.predict(np.array(market_future).reshape(-1, 1))
    
    single_index_regs[ticker] = single_index_reg
    betas[ticker] = single_index_reg.coef_[0][0]
    alphas[ticker] = single_index_reg.intercept_[0]
    
    y_pred = single_index_reg.predict(np.array(market_index_returns[:-10]).reshape(-1, 1))
    residuals = np.array(stock_returns[ticker][10:]) - y_pred.flatten()
    residual_variances[ticker] = np.var(residuals)

# Using Treynor Black model for portfolio optimization

#STEP 1:
# Compute the initial position of each security
weights = {}
for ticker in tickers:
    weights[ticker] = alphas[ticker] / residual_variances[ticker]

#STEP 2:
# Scale the initial positions
scaled_weights = {}
total_weight = sum(weights.values())
for ticker in tickers:
    scaled_weights[ticker] = weights[ticker] / total_weight

#STEP 3:
# Compute the alpha of the active portfolio
alpha_portfolio = sum(scaled_weights[ticker] * alphas[ticker] for ticker in tickers)

#STEP 4:
# Compute the residual variance of active portfolio
residual_variance_portfolio = sum(scaled_weights[ticker] ** 2 * residual_variances[ticker] for ticker in tickers)

#STEP 5:
# Compute the initial position in active portfolio using the values found in first block
initial_position_portfolio = (alpha_portfolio * residual_variance_market) / (residual_variance_portfolio * risk_premium_market)

#STEP 6:
# Compute the beta of active portfolio
beta_portfolio = sum(scaled_weights[ticker] * betas[ticker] for ticker in tickers)

#STEP 7:
# Adjust the initial position in active portfolio
adjusted_position_portfolio = initial_position_portfolio / (1 + (1 - beta_portfolio) * initial_position_portfolio)

#STEP 8:
# Optimal risky portfolio now has weights
final_weight_market = 1-adjusted_position_portfolio
final_weights = {}
for ticker in tickers:
    final_weights[ticker] = adjusted_position_portfolio * scaled_weights[ticker]

print("------------------------------")
print("Final Weights: ")
for ticker, weight in final_weights.items():
    print("Weight {}: {}".format(ticker, weight))
print("------------------------------")

#STEP 9:
# Calculate the risk premium of P (Optimal risky portfolio):
risk_premium_porfolio = (final_weight_market+adjusted_position_portfolio*beta_portfolio)*risk_premium_market + adjusted_position_portfolio*alpha_portfolio
print("Risk premium of portfolio: ", risk_premium_porfolio)

#STEP 10:
# Compute the variance of Portfolio:
portfolio_variance = (final_weight_market+adjusted_position_portfolio*beta_portfolio)*(final_weight_market+adjusted_position_portfolio*beta_portfolio)*residual_variance_market + adjusted_position_portfolio*adjusted_position_portfolio*residual_variance_portfolio
print("Variance of portfolio: ", portfolio_variance)

# Calculate expected returns
returns = expected_returns.returns_from_prices(data['Close'][tickers])
mu = expected_returns.mean_historical_return(returns)

# Calculate covariance matrix
S = risk_models.sample_cov(returns)

returns = expected_returns.returns_from_prices(data['Close'][tickers])
cov_matrix = risk_models.sample_cov(returns)

ef = EfficientFrontier(mu, S)
weights = ef.min_volatility()
cleaned_weights = ef.clean_weights()

# Calculate portfolio return and volatility
portfolio_return = np.dot(returns, np.array(list(weights.values())))
portfolio_volatility = np.sqrt(np.dot(np.array(list(weights.values())), np.dot(cov_matrix, np.array(list(weights.values())))))

# Calculate Sharpe ratio
risk_free_rate = 0.02
sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_volatility

print(f"Sharpe Ratio: {np.mean(np.array(sharpe_ratio))}")

[*********************100%***********************]  16 of 16 completed
------------------------------
Final Weights: 
Weight AAPL: 0.0943377497172858
Weight AMD: 0.052034744518992454
Weight AMZN: 0.033851639904094226
Weight CCJ: 0.044937042508007785
Weight COST: 0.11579466827021821
Weight GOOG: 0.059739532534328135
Weight GS: 0.035565116605628375
Weight JPM: 0.03334419171775533
Weight LLY: 0.12685312988190878
Weight META: 0.026213376953347495
Weight MSFT: 0.09419297715675276
Weight NEE: 0.06203913079553335
Weight PFE: 0.021091155813863446
Weight SAP: 0.027558378254956547
Weight WMT: 0.08907332238300203
------------------------------
Risk premium of portfolio:  0.0009503824521770077
Variance of portfolio:  2.855801305570266e-05
Sharpe Ratio: -0.0002816307573460595


c:\Users\mattg\Desktop\projects\venv\lib\site-packages\pypfopt\expected_returns.py:32: UserWarning: Some returns are NaN. Please check your price data.
  warnings.warn(
c:\Users\mattg\Desktop\projects\venv\lib\site-packages\pypfopt\expected_returns.py:36: UserWarning: Some returns are infinite. Please check your price data.
  warnings.warn(


In [3]:
# Logistic Regression
from pypfopt import plotting
import numpy as np
import pandas as pd
from sklearn.svm import SVR
from pypfopt import expected_returns, risk_models, EfficientFrontier, CovarianceShrinkage
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestRegressor
import datetime
import yfinance as yf

# Define the ticker symbols for the market index and individual assets
market_index_ticker = "^GSPC"  # S&P 500 index
tickers = ['AAPL', 'AMD', 'AMZN', 'CCJ', 'COST', 'GOOG', 'GS', 'JPM', 'LLY', 'META', 'MSFT', 'NEE', 'PFE', 'SAP', 'WMT']

# Define the start and end dates for the historical data
end_date = datetime.datetime.now()
start_date = end_date - datetime.timedelta(days=5*365)

# Fetch the historical data using yfinance
data = yf.download([market_index_ticker] + tickers, start=start_date, end=end_date)

# Returns of stocks and market
market_index_returns = ((data['Close'][market_index_ticker] / data['Close'][market_index_ticker].shift(1))-1).dropna().tolist()
stock_returns = {}
for ticker in tickers:
    stock_returns[ticker] = ((data['Close'][ticker] / data['Close'][ticker].shift(1))-1).dropna().tolist()

# Future market returns prediction using logistic regression
market_x = np.array(market_index_returns[:-10]).reshape(-1, 1)
market_y = np.array(market_index_returns[10:])

reg = LogisticRegression()
reg.fit(market_x, np.sign(market_y))
market_future = reg.predict(np.array(market_index_returns[-10:]).reshape(-1, 1))

# Fitting single index model regression on market return and stock return
single_index_regs = {}
betas = {}
alphas = {}
residual_variances = {}

for ticker in tickers:
    single_index_reg = LinearRegression()
    single_index_reg.fit(np.array(market_index_returns[:-10]).reshape(-1, 1), np.array(stock_returns[ticker][10:]).reshape(-1, 1))
    stock_future = single_index_reg.predict(np.array(market_future).reshape(-1, 1))
    
    single_index_regs[ticker] = single_index_reg
    betas[ticker] = single_index_reg.coef_[0][0]
    alphas[ticker] = single_index_reg.intercept_[0]
    
    y_pred = single_index_reg.predict(np.array(market_index_returns[:-10]).reshape(-1, 1))
    residuals = np.array(stock_returns[ticker][10:]) - y_pred.flatten()
    residual_variances[ticker] = np.var(residuals)

# Using Treynor Black model for portfolio optimization

#STEP 1:
# Compute the initial position of each security
weights = {}
for ticker in tickers:
    weights[ticker] = alphas[ticker] / residual_variances[ticker]

#STEP 2:
# Scale the initial positions
scaled_weights = {}
total_weight = sum(weights.values())
for ticker in tickers:
    scaled_weights[ticker] = weights[ticker] / total_weight

#STEP 3:
# Compute the alpha of the active portfolio
alpha_portfolio = sum(scaled_weights[ticker] * alphas[ticker] for ticker in tickers)

#STEP 4:
# Compute the residual variance of active portfolio
residual_variance_portfolio = sum(scaled_weights[ticker] ** 2 * residual_variances[ticker] for ticker in tickers)

#STEP 5:
# Compute the initial position in active portfolio
initial_position_portfolio = (alpha_portfolio * residual_variance_market) / (residual_variance_portfolio * risk_premium_market)

#STEP 6:
# Compute the beta of active portfolio
beta_portfolio = sum(scaled_weights[ticker] * betas[ticker] for ticker in tickers)

#STEP 7:
# Adjust the initial position in active portfolio
adjusted_position_portfolio = initial_position_portfolio / (1 + (1 - beta_portfolio) * initial_position_portfolio)

#STEP 8:
# Optimal risky portfolio now has weights
final_weight_market = 1-adjusted_position_portfolio
final_weights = {}
for ticker in tickers:
    final_weights[ticker] = adjusted_position_portfolio * scaled_weights[ticker]

print("------------------------------")
print("Final Weights: ")
for ticker, weight in final_weights.items():
    print("Weight {}: {}".format(ticker, weight))
print("------------------------------")

#STEP 9:
# Calculate the risk premium of P (Optimal risky portfolio):
risk_premium_porfolio = (final_weight_market+adjusted_position_portfolio*beta_portfolio)*risk_premium_market + adjusted_position_portfolio*alpha_portfolio
print("Risk premium of portfolio: ", risk_premium_porfolio)

#STEP 10:
# Compute the variance of Portfolio:
portfolio_variance = (final_weight_market+adjusted_position_portfolio*beta_portfolio)*(final_weight_market+adjusted_position_portfolio*beta_portfolio)*residual_variance_market + adjusted_position_portfolio*adjusted_position_portfolio*residual_variance_portfolio
print("Variance of portfolio: ", portfolio_variance)

# Calculate expected returns
returns = expected_returns.returns_from_prices(data['Close'][tickers])
mu = expected_returns.mean_historical_return(returns)

# Calculate covariance matrix
S = risk_models.sample_cov(returns)

returns = expected_returns.returns_from_prices(data['Close'][tickers])
cov_matrix = risk_models.sample_cov(returns)

ef = EfficientFrontier(mu, S)
weights = ef.min_volatility()
cleaned_weights = ef.clean_weights()

# Calculate portfolio return and volatility
portfolio_return = np.dot(returns, np.array(list(weights.values())))
portfolio_volatility = np.sqrt(np.dot(np.array(list(weights.values())), np.dot(cov_matrix, np.array(list(weights.values())))))

# Calculate Sharpe ratio
risk_free_rate = 0.02
sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_volatility

print(f"Sharpe Ratio: {np.mean(np.array(sharpe_ratio))}")

[*********************100%***********************]  16 of 16 completed
------------------------------
Final Weights: 
Weight AAPL: 0.0943377497172858
Weight AMD: 0.052034744518992454
Weight AMZN: 0.033851639904094226
Weight CCJ: 0.044937042508007785
Weight COST: 0.11579466827021821
Weight GOOG: 0.059739532534328135
Weight GS: 0.035565116605628375
Weight JPM: 0.03334419171775533
Weight LLY: 0.12685312988190878
Weight META: 0.026213376953347495
Weight MSFT: 0.09419297715675276
Weight NEE: 0.06203913079553335
Weight PFE: 0.021091155813863446
Weight SAP: 0.027558378254956547
Weight WMT: 0.08907332238300203
------------------------------
Risk premium of portfolio:  0.0009503824521770077
Variance of portfolio:  2.855801305570266e-05
Sharpe Ratio: -0.0002816307573460595


c:\Users\mattg\Desktop\projects\venv\lib\site-packages\pypfopt\expected_returns.py:32: UserWarning: Some returns are NaN. Please check your price data.
  warnings.warn(
c:\Users\mattg\Desktop\projects\venv\lib\site-packages\pypfopt\expected_returns.py:36: UserWarning: Some returns are infinite. Please check your price data.
  warnings.warn(


In [4]:
# SVM
from pypfopt import plotting
import numpy as np
import pandas as pd
from sklearn.svm import SVR
from pypfopt import expected_returns, risk_models, EfficientFrontier
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import datetime
import yfinance as yf
import warnings

warnings.filterwarnings("ignore")

# Define the ticker symbols for the market index and individual assets
market_index_ticker = "^GSPC"  # S&P 500 index
tickers = ['AAPL', 'AMD', 'AMZN', 'CCJ', 'COST', 'GOOG', 'GS', 'JPM', 'LLY', 'META', 'MSFT', 'NEE', 'PFE', 'SAP', 'WMT']

# Define the start and end dates for the historical data
end_date = datetime.datetime.now()
start_date = end_date - datetime.timedelta(days=5*365)

# Fetch the historical data using yfinance
data = yf.download([market_index_ticker] + tickers, start=start_date, end=end_date)

# Returns of stocks and market
market_index_returns = ((data['Close'][market_index_ticker] / data['Close'][market_index_ticker].shift(1))-1).dropna().tolist()
stock_returns = {}
for ticker in tickers:
    stock_returns[ticker] = ((data['Close'][ticker] / data['Close'][ticker].shift(1))-1).dropna().tolist()

# Future market returns prediction using Support Vector Regression (SVR)
market_x = np.array(market_index_returns[:-10]).reshape(-1, 1)
market_y = np.array(market_index_returns[10:])

reg = SVR(kernel='linear')
reg.fit(market_x, market_y)
market_future = reg.predict(np.array(market_index_returns[-10:]).reshape(-1, 1))

# Fitting single index model regression on market return and stock return
single_index_regs = {}
betas = {}
alphas = {}
residual_variances = {}

for ticker in tickers:
    single_index_reg = SVR(kernel='linear')
    single_index_reg.fit(np.array(market_index_returns[:-10]).reshape(-1, 1), np.array(stock_returns[ticker][10:]).reshape(-1, 1))
    stock_future = single_index_reg.predict(np.array(market_future).reshape(-1, 1))
    
    single_index_regs[ticker] = single_index_reg
    betas[ticker] = single_index_reg.coef_[0][0]
    alphas[ticker] = single_index_reg.intercept_[0]
    
    y_pred = single_index_reg.predict(np.array(market_index_returns[:-10]).reshape(-1, 1))
    residuals = np.array(stock_returns[ticker][10:]) - y_pred.flatten()
    residual_variances[ticker] = np.var(residuals)

# Using Treynor Black model for portfolio optimization

#STEP 1:
# Compute the initial position of each security
weights = {}
for ticker in tickers:
    weights[ticker] = alphas[ticker] / residual_variances[ticker]

#STEP 2:
# Scale the initial positions
scaled_weights = {}
total_weight = sum(weights.values())
for ticker in tickers:
    scaled_weights[ticker] = weights[ticker] / total_weight

#STEP 3:
# Compute the alpha of the active portfolio
alpha_portfolio = sum(scaled_weights[ticker] * alphas[ticker] for ticker in tickers)

#STEP 4:
# Compute the residual variance of active portfolio
residual_variance_portfolio = sum(scaled_weights[ticker] ** 2 * residual_variances[ticker] for ticker in tickers)

#STEP 5:
# Compute the initial position in active portfolio
initial_position_portfolio = (alpha_portfolio * residual_variance_market) / (residual_variance_portfolio * risk_premium_market)

#STEP 6:
# Compute the beta of active portfolio
beta_portfolio = sum(scaled_weights[ticker] * betas[ticker] for ticker in tickers)

#STEP 7:
# Adjust the initial position in active portfolio
adjusted_position_portfolio = initial_position_portfolio / (1 + (1 - beta_portfolio) * initial_position_portfolio)

#STEP 8:
# Optimal risky portfolio now has weights
final_weight_market = 1-adjusted_position_portfolio
final_weights = {}
for ticker in tickers:
    final_weights[ticker] = adjusted_position_portfolio * scaled_weights[ticker]

print("------------------------------")
print("Final Weights: ")
for ticker, weight in final_weights.items():
    print("Weight {}: {}".format(ticker, weight))
print("------------------------------")

#STEP 9:
# Calculate the risk premium of P (Optimal risky portfolio):
risk_premium_porfolio = (final_weight_market+adjusted_position_portfolio*beta_portfolio)*risk_premium_market + adjusted_position_portfolio*alpha_portfolio
print("Risk premium of portfolio: ", risk_premium_porfolio)

#STEP 10:
# Compute the variance of Portfolio:
portfolio_variance = (final_weight_market+adjusted_position_portfolio*beta_portfolio)*(final_weight_market+adjusted_position_portfolio*beta_portfolio)*residual_variance_market + adjusted_position_portfolio*adjusted_position_portfolio*residual_variance_portfolio
print("Variance of portfolio: ", portfolio_variance)

# Calculate expected returns
returns = expected_returns.returns_from_prices(data['Close'][tickers])
mu = expected_returns.mean_historical_return(returns)

# Calculate covariance matrix
S = risk_models.sample_cov(returns)

returns = expected_returns.returns_from_prices(data['Close'][tickers])
cov_matrix = risk_models.sample_cov(returns)

ef = EfficientFrontier(mu, S)
weights = ef.min_volatility()
cleaned_weights = ef.clean_weights()

# Calculate portfolio return and volatility
portfolio_return = np.dot(returns, np.array(list(weights.values())))
portfolio_volatility = np.sqrt(np.dot(np.array(list(weights.values())), np.dot(cov_matrix, np.array(list(weights.values())))))

# Calculate Sharpe ratio
risk_free_rate = 0.02
sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_volatility

print(f"Sharpe Ratio: {np.mean(np.array(sharpe_ratio))}")


[*********************100%***********************]  16 of 16 completed
------------------------------
Final Weights: 
Weight AAPL: 0.01372871605871721
Weight AMD: 0.06228591389132299
Weight AMZN: 0.2530508303043935
Weight CCJ: 0.0015982195867278264
Weight COST: -0.27013858175892347
Weight GOOG: -0.025975676145200535
Weight GS: -0.12755804945885774
Weight JPM: 0.11075234976556139
Weight LLY: 0.3860122543684306
Weight META: 0.03423592340509261
Weight MSFT: -0.03546746378796864
Weight NEE: 0.13986163649162614
Weight PFE: 0.4161326114751699
Weight SAP: -0.016339611311835117
Weight WMT: 0.07669692375693718
------------------------------
Risk premium of portfolio:  0.024037788460425643
Variance of portfolio:  0.00018042506963689722
Sharpe Ratio: -0.0002816307573460595
